In [6]:
# Step 1: Install Dependencies
# Install required libraries
# pip install gym stable-baselines3 pandas numpy

# Step 2: Import Libraries
import gym
import numpy as np
import pandas as pd
from gym import spaces
from stable_baselines3 import PPO

# Step 3: Create the Custom Environment for Bovine Respiratory Disease Prediction
class BovineRespiratoryDiseaseEnv(gym.Env):
    def __init__(self, data):
        super(BovineRespiratoryDiseaseEnv, self).__init__()
        
        # Load the dataset containing cattle health metrics
        self.data = data
        self.current_step = 0
        
        # Action space: Decide on health intervention (e.g., increase observation, administer medication)
        self.action_space = spaces.Discrete(2)  # 0 = No intervention, 1 = Intervention
        
        # Observation space: Use the cattle health data features (e.g., temperature, heart rate)
        self.observation_space = spaces.Box(low=0, high=1, shape=(data.shape[1] - 1,), dtype=np.float32)
    
    def reset(self):
        # Reset the environment to the initial state
        self.current_step = 0
        return self.data.iloc[self.current_step, :-1].values
    
    def step(self, action):
        # Get current observation
        observation = self.data.iloc[self.current_step, :-1].values
        
        # Calculate reward: higher rewards for correctly predicting intervention (preventing BRD)
        reward = 0
        if action == 1 and self.data.iloc[self.current_step]['BRD_Total'] == 1:  # Disease present, intervention made
            reward = 10  # Positive reward for correct intervention
        elif action == 0 and self.data.iloc[self.current_step]['BRD_Total'] == 0:  # No disease, no intervention
            reward = 5  # Positive reward for no action when no disease present
        else:
            reward = -5  # Negative reward for incorrect action
        
        self.current_step += 1
        done = self.current_step >= len(self.data)
        
        return observation, reward, done, {}

# Step 4: Load Your Dataset from a CSV File
# Example: 'cattle_health_data.csv' should be replaced with your actual file path
file_path = r"fulldataset.xlsx"
# Load the dataset
data = pd.read_excel(file_path, engine='openpyxl')

# Step 5: Prepare Data for Training
# Fill NaN values in the 'BRD_Total' column with 0
data['BRD_Total'] = data['BRD_Total'].fillna(0).astype(int)

# Normalize the data (optional but helpful for RL)
# Make sure to normalize the feature columns (except for the target column 'BRD_Total')
data_normalized = data.copy()

# Select only numeric columns for normalization
numeric_columns = data_normalized.select_dtypes(include=[np.number]).columns
data_normalized[numeric_columns] = data_normalized[numeric_columns] / (data_normalized[numeric_columns].max() + 1e-8)  # Avoid division by zero

# Ensure the environment receives only numeric data
data_normalized = data_normalized[numeric_columns]

# Check for NaN values in the data
if data_normalized.isnull().sum().any():
    print("NaN values detected! Replacing them.")
    data_normalized = data_normalized.fillna(0)

# Step 6: Train the RL Model
# Create the custom environment
env = BovineRespiratoryDiseaseEnv(data_normalized)

# Initialize the PPO model with a lower learning rate to avoid NaNs
learning_rate = 0.0001
training_rate = 10000
model = PPO('MlpPolicy', env, learning_rate=learning_rate, verbose=1)

# Train the model
model.learn(total_timesteps=training_rate)

# Step 7: Evaluate the Model and Calculate Accuracy
# Initialize variables for tracking correct and total predictions
correct_predictions = 0
total_predictions = 0

# Test the trained agent
obs = env.reset()
done = False

# Evaluate over the entire dataset
while not done:
    action, _states = model.predict(obs)
    
    # Compare predicted action to actual disease state to determine correctness
    if (action == 1 and env.data.iloc[env.current_step - 1]['BRD_Total'] == 1) or \
       (action == 0 and env.data.iloc[env.current_step - 1]['BRD_Total'] == 0):
        correct_predictions += 1
    total_predictions += 1
    
    obs, reward, done, _ = env.step(action)

# Calculate accuracy
accuracy = correct_predictions / total_predictions * 100
print(f"Model Accuracy: {accuracy:.2f}%")

# Step 8: Make a Table of Results
# Create a table with the model's learning rate, training rate, and accuracy
results = {
    "Learning Rate": [learning_rate],
    "Training Rate": [training_rate],
    "Model Accuracy (%)": [accuracy],
    "Correct Predictions": [correct_predictions],
    "Total Predictions": [total_predictions]
}

results_table = pd.DataFrame(results)
print("\nModel Training Results Table:")
print(results_table)

# Step 9: Making Predictions with the Trained Model
# Example: Make a prediction for the next cattle observation
test_observation = np.array([0.6, 0.7, 0.8, 0.5])  # Replace with actual data

# Ensure the test observation has the correct shape (e.g., 32 features)
# If your data has fewer than 32 features, you might need to pad or extend this observation
# Here we manually extend it to match the expected size.
test_observation = np.pad(test_observation, (0, 32 - len(test_observation)), 'constant')

# Reshape the observation to add a batch dimension (1, 32)
test_observation = test_observation.reshape(1, -1)

# Get the prediction
action, _states = model.predict(test_observation)

# Output action
if action == 1:
    print("Intervene: Cattle at risk of BRD.")
else:
    print("No intervention needed.")


NaN values detected! Replacing them.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\HP DRAGON FLY\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 1488 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 900         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007556426 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.69       |
|    explained_variance   | -0.000185   |
|    learning_rate        | 0.0001      |
|    loss                 | 2.41e+03    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00277    |
|    value_loss           | 5.35e+03    |
-----------------------------------------
----------------------------------

In [5]:
import gym
import numpy as np
import pandas as pd
from gym import spaces
from stable_baselines3 import PPO

# Step 3: Create the Custom Environment for Bovine Respiratory Disease Prediction
class BovineRespiratoryDiseaseEnv(gym.Env):
    def __init__(self, data):
        super(BovineRespiratoryDiseaseEnv, self).__init__()
        
        # Load the dataset containing cattle health metrics
        self.data = data
        self.current_step = 0
        
        # Action space: Decide on health intervention (e.g., increase observation, administer medication)
        self.action_space = spaces.Discrete(2)  # 0 = No intervention, 1 = Intervention
        
        # Observation space: Use the cattle health data features (e.g., temperature, heart rate)
        self.observation_space = spaces.Box(low=0, high=1, shape=(data.shape[1] - 1,), dtype=np.float32)
    
    def reset(self):
        # Reset the environment to the initial state
        self.current_step = 0
        return self.data.iloc[self.current_step, :-1].values
    
    def step(self, action):
        # Get current observation
        observation = self.data.iloc[self.current_step, :-1].values
        
        # Calculate reward: higher rewards for correctly predicting intervention (preventing BRD)
        reward = 0
        if action == 1 and self.data.iloc[self.current_step]['clade'] == 1:  # Disease present, intervention made
            reward = 10  # Positive reward for correct intervention
        elif action == 0 and self.data.iloc[self.current_step]['clade'] == 0:  # No disease, no intervention
            reward = 5  # Positive reward for no action when no disease present
        else:
            reward = -5  # Negative reward for incorrect action
        
        self.current_step += 1
        done = self.current_step >= len(self.data)
        
        return observation, reward, done, {}

# Step 4: Load Your Dataset from a CSV File
# Example: 'cattle_health_data.csv' should be replaced with your actual file path

# Load the dataset
data = pd.read_csv("global_meta.csv")  # Correct method to read a CSV file

# Step 5: Prepare Data for Training
# Fill NaN values in the 'clade' column with 0
data['clade'] = data['clade'].fillna(0).astype(int)

# Normalize the data (optional but helpful for RL)
# Make sure to normalize the feature columns (except for the target column 'clade')
data_normalized = data.copy()

# Select only numeric columns for normalization
numeric_columns = data_normalized.select_dtypes(include=[np.number]).columns
data_normalized[numeric_columns] = data_normalized[numeric_columns] / (data_normalized[numeric_columns].max() + 1e-8)  # Avoid division by zero

# Ensure the environment receives only numeric data
data_normalized = data_normalized[numeric_columns]

# Check for NaN values in the data
if data_normalized.isnull().sum().any():
    print("NaN values detected! Replacing them.")
    data_normalized = data_normalized.fillna(0)

# Step 6: Train the RL Model
# Create the custom environment
env = BovineRespiratoryDiseaseEnv(data_normalized)

# Initialize the PPO model with a lower learning rate to avoid NaNs
model = PPO('MlpPolicy', env, learning_rate=0.0001, verbose=1)

# Train the model
model.learn(total_timesteps=10000)

# Step 7: Evaluate the Model and Calculate Accuracy
# Initialize variables for tracking correct and total predictions
correct_predictions = 0
total_predictions = 0

# Test the trained agent
obs = env.reset()
done = False

while not done:
    action, _states = model.predict(obs)
    
    # Compare predicted action to actual disease state to determine correctness
    if (action == 1 and env.data.iloc[env.current_step - 1]['clade'] == 1) or \
       (action == 0 and env.data.iloc[env.current_step - 1]['clade'] == 0):
        correct_predictions += 1
    total_predictions += 1
    
    obs, reward, done, _ = env.step(action)

# Calculate accuracy
accuracy = correct_predictions / total_predictions
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Step 8: Making Predictions with the Trained Model
# Example: Make a prediction for the next cattle observation
test_observation = np.array([0.6, 0.7, 0.8, 0.5])  # Replace with actual data

# Ensure the test observation has the correct shape (should be 6 features, not 32)
test_observation = np.array([0.6, 0.7, 0.8, 0.5, 0.3, 0.2])  # Replace with actual data

# Reshape to match the expected input (1, 6) since it's a single sample
test_observation = test_observation.reshape(1, -1)

# Get the prediction
action, _states = model.predict(test_observation)

# Output action
if action == 1:
    print("Intervene: Cattle at risk of BRD.")
else:
    print("No intervention needed.")


NaN values detected! Replacing them.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\HP DRAGON FLY\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.72e+03 |
|    ep_rew_mean     | -220     |
| time/              |          |
|    fps             | 1474     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.72e+03    |
|    ep_rew_mean          | 365         |
| time/                   |             |
|    fps                  | 1031        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012246516 |
|    clip_fraction        | 0.0993      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.688      |
|    explained_variance   | -0.000528   |
|    learning_rate        | 0.